# CS 109A/STAT 121A/AC 209A/CSCI E-109A: 
# Final Project: Group 31

**Harvard University**<br/>
**Fall 2017**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
#import pydotplus
#import io
from sklearn.tree import export_graphviz
from IPython.display import Image
from IPython.display import display
from sklearn.metrics import accuracy_score
import datetime as DT
import seaborn as sns
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.utils import resample
from scipy import stats

%matplotlib inline

C:\Users\IhnSik\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Spotify Predictors Only

## Shotgun Phase

In [2]:
# read csv of spotify predictors
df = pd.read_csv('./spotify_preds_csv2.csv')
df = df.dropna()
df = df.drop(['Unnamed: 0'], 1)

In [3]:
# split data
np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.75
data_train = df[msk]
data_test = df[~msk]

In [4]:
column_headers = list(data_train.columns.values)

In [5]:
# get variables
X_train = data_train.iloc[:,6:]
y_train = data_train.iloc[:, 1]

X_test = data_test.iloc[:,6:]
y_test = data_test.iloc[:, 1]

In [6]:
# Linear Regression
X_train2 = sm.add_constant(X_train.values)
model = sm.OLS(y_train.values, X_train2)
results = model.fit()

y_hat_train = results.predict(X_train2)

# test case
X_test2 = sm.add_constant(X_test.values)
y_hat_test = results.predict(X_test2)

r2_score_train = r2_score(y_train, y_hat_train) 
r2_score_test = r2_score(y_test, y_hat_test)

print('R^2 Values for Train, Test Using Linear Regression:', r2_score_train, r2_score_test)


R^2 Values for Train, Test Using Linear Regression: 0.169149915072 0.179534112657


In [7]:
# polynomial term regression
from sklearn import linear_model
r2_train_poly = []
r2_test_poly = []

# make dataframes to add polynomial terms to
cont = ['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_num_artists', 'avg_num_markets']
X_binary_only = X_train.drop(['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_num_artists', 'avg_num_markets'], axis=1)
X_test_bin_only = X_test.drop(['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_num_artists', 'avg_num_markets'], axis=1)
X_poly = X_binary_only.copy()
X_test_poly = X_test_bin_only.copy()

X_poly_test_all = X_test.copy()
X_poly_train_all = X_train.copy()

# function to create and add polynomial terms to dataframe
def add_poly_features(train, test, poly_train, poly_test, polylist):
    for col in polylist:
        for i in range(2,4):
            poly_train[col + '_' + str(i)] = train[col]**i
            poly_test[col + '_' + str(i)] = test[col]**i

add_poly_features(X_train, X_test, X_poly_train_all, X_poly_test_all,cont)

# polynomial term regression 
poly_regression_model = linear_model.LinearRegression(fit_intercept=False)
poly_regression_model.fit(X_poly_train_all, y_train)

y_hat_train = poly_regression_model.predict(X_poly_train_all)
y_hat_test = poly_regression_model.predict(X_poly_test_all)


r2_train_poly.append( r2_score(y_train, y_hat_train))
r2_test_poly.append( r2_score(y_test, y_hat_test))

print('R^2 Values for Train, Test Using Polynomial Regression:', r2_train_poly, r2_test_poly)


R^2 Values for Train, Test Using Polynomial Regression: [0.22894924317795406] [0.19789980160737619]


In [8]:
# knn regression
from sklearn.neighbors import KNeighborsRegressor
# try multiple K's
K = [1, 2, 4,8, 10, 50, 100, 250, 500, 600, 700, 800, 900, 1000]
r2_test_knn = []
r2_train_knn = []

# try each different K and calculate R^2
for i,k in enumerate(K): 
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(X_train, y_train)
    predicted_pickups_train = knn_model.predict(X_train)
    predicted_pickups = knn_model.predict(X_test)

    r2_train_knn.append( r2_score(y_train, predicted_pickups_train))
    r2_test_knn.append( r2_score(y_test, predicted_pickups))
    
    
print('R^2 Values for Train Using KNN Regression:', r2_train_knn)
print('\n')
print('R^2 Values for Test Using KNN Regression:', r2_test_knn)


R^2 Values for Train Using KNN Regression: [0.99999999826605368, 0.67361557405130701, 0.26014861730618322, 0.14666628473394638, 0.12441822659635149, 0.042620591885406456, 0.029105850221909146, 0.022963572284714795, 0.013139851669411118, 0.011592111214729361, 0.0092869321588529008, 0.0077948862622220227, 0.0073745784743184384, 0.0056233810101773418]


R^2 Values for Test Using KNN Regression: [-0.75715473073802242, -0.25301680053630093, -0.30348630520483377, -0.094250067982748709, -0.12328060001080909, -0.021899414008623497, -0.013522963401996657, 0.0045474078918134042, 0.0031360746980091392, 0.0017065831673064302, 0.0013485495695024774, 0.0014863803700982947, 0.00016419258602440312, -0.00075057625934760175]


In [9]:
# random forest regression
from sklearn.ensemble import RandomForestRegressor
r2_train_rf = []
r2_test_rf = []

# check multiple depths to see which depth is best
for i in range(1, 20):
    rf_reg = RandomForestRegressor(max_depth=i)
    rf_reg.fit(X_train, y_train)

    rf_yhat_train = rf_reg.predict(X_train)
    rf_yhat_test = rf_reg.predict(X_test)

    r2_train_rf.append( r2_score(y_train, rf_yhat_train))
    r2_test_rf.append( r2_score(y_test, rf_yhat_test))

print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf)

R^2 Values for Train Using Random Forest Regression: [0.49720296756434346, 0.61270131485010926, 0.68325744938432675, 0.67393279843134068, 0.72210693690258476, 0.76032229514454808, 0.82802319507749478, 0.89219881258399536, 0.8249767766794609, 0.91741728542031664, 0.84918705143797346, 0.80188428891226848, 0.80126643347813964, 0.79903329718948535, 0.83801613983253065, 0.8615756848017454, 0.92325586257123493, 0.8403246954524074, 0.93468703982697143]


R^2 Values for Test Using Random Forest Regression: [0.065688530132135381, 0.11686894621887656, 0.24312942359278267, 0.25580666456779466, 0.26659964595678221, 0.36562212711641229, 0.30701531979506791, 0.29929058898637073, 0.35091131308104861, 0.24928732729333758, 0.26858970989981312, 0.2692000957426397, 0.22855920080718684, 0.34740277432435152, 0.27613072857891741, 0.21208542049379497, 0.32475518293240679, 0.2865265024936684, 0.28830233748470802]


In [10]:
# get best depth
import operator
index, value = max(enumerate(r2_test_rf), key=operator.itemgetter(1))
best_depth = index+1
print('Best Depth for Random Forest Tree Depth:', best_depth )

Best Depth for Random Forest Tree Depth: 6


## Fine Tuning Phase

### Ridge and Lasso for Polynomial Term Regression

In [11]:
# Ridge Regression on Polynomial Term Regression
lambdas = [.001,.005,1,5,10,50,100,500,1000]    
ridge = RidgeCV(alphas=lambdas, fit_intercept=False, normalize=True, cv=10)
ridge.fit(X_poly_train_all, y_train)
print("Ridge train R2: ", ridge.score(X_poly_train_all, y_train))
print('Ridge test R2', ridge.score(X_poly_test_all, y_test))


Ridge train R2:  0.311635637957
Ridge test R2 0.263143609467


In [12]:
# Lasso Regression on Polynomial Term Regression
lasso = LassoCV(alphas=lambdas, fit_intercept=False, normalize=True, cv=10)
lasso.fit(X_poly_train_all, y_train)
print("Lasso train R2: ", lasso.score(X_poly_train_all, y_train))
print('Lasso test R2', lasso.score(X_poly_test_all, y_test))

C:\Users\IhnSik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso train R2:  0.309110797466
Lasso test R2 0.268284143325


C:\Users\IhnSik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [13]:
# Fine Tuning Random Forest: Initial Run with max_depth set to the optimal depth from shotgun approach
r2_train_rf = []
r2_test_rf = []

rf_reg = RandomForestRegressor(max_depth=best_depth)
rf_reg.fit(X_train, y_train)

rf_yhat_train = rf_reg.predict(X_train)
rf_yhat_test = rf_reg.predict(X_test)

r2_train_rf.append( r2_score(y_train, rf_yhat_train))
r2_test_rf.append( r2_score(y_test, rf_yhat_test))

print('R^2 Values for Train, Test Using Random Forest Regression:', r2_train_rf, r2_test_rf)

R^2 Values for Train, Test Using Random Forest Regression: [0.82293769375947434] [0.31770676189595115]


In [14]:
# step 1: fine tune the number of trees
r2_train_rf_trees = []
r2_test_rf_trees = []

# create list of tree numbers we will test
trees = [2**x for x in range(8)]  # 2, 4, 8, 16, 32, ... 

# test the tree numbers keeping max_depth at 9
for n_trees in trees:
    rf = RandomForestRegressor(n_estimators=n_trees, max_depth=best_depth, max_features='auto')
    rf.fit(X_train, y_train)

    rf_yhat_train = rf.predict(X_train)
    rf_yhat_test = rf.predict(X_test)

    r2_train_rf_trees.append(r2_score(y_train, rf_yhat_train))
    r2_test_rf_trees.append(r2_score(y_test, rf_yhat_test))
    
print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf_trees)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf_trees)

R^2 Values for Train Using Random Forest Regression: [0.54390777801011136, 0.74915188465836335, 0.68857360331932815, 0.66173815567878069, 0.83747904942944829, 0.78575995159604661, 0.80893574377847954, 0.78342305887328823]


R^2 Values for Test Using Random Forest Regression: [0.15396519201018743, 0.16246121247554413, 0.24043659131667328, 0.32963429252432519, 0.2785371021681623, 0.30173022605065747, 0.25964668576958727, 0.28956393856307283]


In [15]:
# get best number of trees
index, value = max(enumerate(r2_test_rf_trees), key=operator.itemgetter(1))
best_tree = trees[index]
print('RandomForest Best Number of Trees:',best_tree)
print('R^2 Value:',value)

RandomForest Best Number of Trees: 8
R^2 Value: 0.329634292524


In [16]:
# step 2: fine tune the number of predictors used
r2_train_rf_feat = []
r2_test_rf_feat = []

# 23 is the maximum number of predictors we have
for i in range(1, 23):
    rf = RandomForestRegressor(n_estimators=32, max_depth=9, max_features=i)
    rf.fit(X_train, y_train)

    rf_yhat_train = rf.predict(X_train)
    rf_yhat_test = rf.predict(X_test)

    r2_train_rf_feat.append(r2_score(y_train, rf_yhat_train))
    r2_test_rf_feat.append(r2_score(y_test, rf_yhat_test))
    
print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf_feat)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf_feat)

R^2 Values for Train Using Random Forest Regression: [0.81749611921464682, 0.77321846476908007, 0.78819462911672866, 0.816925271008485, 0.87678836077399691, 0.83477997990957764, 0.87500109687797534, 0.87913004926886285, 0.88820112466808265, 0.85992854321804202, 0.8924926560477211, 0.87891394254210442, 0.82457090459714277, 0.8451926191871838, 0.91019444568084384, 0.85959105566094851, 0.86102047310496765, 0.86257570783480808, 0.85157693727582018, 0.84752315378962051, 0.86590999354425557, 0.86833763456460822]


R^2 Values for Test Using Random Forest Regression: [0.21009750906712221, 0.21443955123969971, 0.20966451049282442, 0.19029712386338726, 0.24225359205778041, 0.27066520483498713, 0.25029078641046187, 0.23058538778632154, 0.23860257884132918, 0.26594228903196993, 0.29036814295350932, 0.24813294051505774, 0.26720846088508354, 0.31264175037730524, 0.2455125911463204, 0.27811986532747734, 0.27167901959511009, 0.23134570775926233, 0.27702489305262901, 0.31034805785537567, 0.299271597323

In [17]:
# get best number of predictors
import operator
index, value = max(enumerate(r2_test_rf_feat), key=operator.itemgetter(1))
print('RandomForest Number of Predictors for Best value:',index+1)
print('Best R^2 Value Using Only Spotify Predictors:',value)

RandomForest Number of Predictors for Best value: 14
Best R^2 Value Using Only Spotify Predictors: 0.312641750377


# Combined Predictors

In [18]:
# read csv that combines previous csv from spotify with few additional columns of additional predictors
df = pd.read_csv('./final_dataset.csv')
df = df.dropna()

In [19]:
# split data
np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.75
data_train = df[msk]
data_test = df[~msk]

In [20]:
# get predictor list
column_headers = list(data_train.columns.values)


## Shotgun Phase

In [21]:
# get variables
X_train = data_train.iloc[:,6:]
y_train = data_train.iloc[:, 1]

X_test = data_test.iloc[:,6:]
y_test = data_test.iloc[:, 1]

In [22]:
# Linear Regression
X_train2 = sm.add_constant(X_train.values)
model = sm.OLS(y_train.values, X_train2)
results = model.fit()

y_hat_train = results.predict(X_train2)

# test case
X_test2 = sm.add_constant(X_test.values)
y_hat_test = results.predict(X_test2)

r2_score_train = r2_score(y_train, y_hat_train) 
r2_score_test = r2_score(y_test, y_hat_test)

print('R^2 Values for Train, Test Using Linear Regression:', r2_score_train, r2_score_test)


R^2 Values for Train, Test Using Linear Regression: 0.188983912582 0.12729231813


In [23]:
# Polynomial Term Regression
from sklearn import linear_model
r2_train_poly = []
r2_test_poly = []

# set up dataframe to add polynomial terms to
cont = ['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_danceability', 'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_duration_ms', 'avg_num_artists', 'avg_num_markets']
X_binary_only = X_train.drop(['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_danceability', 'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_duration_ms', 'avg_num_artists', 'avg_num_markets'], axis=1)
X_test_bin_only = X_test.drop(['avg_album_popularity', 'avg_album_release_year', 'avg_artist_popularity', 'avg_artist_followers', 'num_tracks', 'avg_song_popularity', 'avg_danceability', 'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_duration_ms', 'avg_num_artists', 'avg_num_markets'], axis=1)
X_poly = X_binary_only.copy()
X_test_poly = X_test_bin_only.copy()

X_poly_test_all = X_test.copy()
X_poly_train_all = X_train.copy()

# function to create and add polynomial terms to dataframe
def add_poly_features(train, test, poly_train, poly_test, polylist):
    for col in polylist:
        for i in range(2,4):
            poly_train[col + '_' + str(i)] = train[col]**i
            poly_test[col + '_' + str(i)] = test[col]**i

add_poly_features(X_train, X_test, X_poly_train_all, X_poly_test_all,cont)

# regress and calculate R^2
poly_regression_model = linear_model.LinearRegression(fit_intercept=False)
poly_regression_model.fit(X_poly_train_all, y_train)

y_hat_train = poly_regression_model.predict(X_poly_train_all)
y_hat_test = poly_regression_model.predict(X_poly_test_all)

r2_train_poly.append( r2_score(y_train, y_hat_train))
r2_test_poly.append( r2_score(y_test, y_hat_test))

print('R^2 Values for Train, Test Using Polynomial Regression:', r2_train_poly, r2_test_poly)


R^2 Values for Train, Test Using Polynomial Regression: [0.22310947907083678] [0.19655342308010126]


In [24]:
# KNN Regression: test different K's
from sklearn.neighbors import KNeighborsRegressor
K = [1, 2, 4,8, 10, 50, 100, 250, 500, 600, 700, 800, 900, 1000]

r2_test_knn = []
r2_train_knn = []

# test different k's and regress
for i,k in enumerate(K): 
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(X_train, y_train)
    predicted_pickups_train = knn_model.predict(X_train)
    predicted_pickups = knn_model.predict(X_test)

    r2_train_knn.append( r2_score(y_train, predicted_pickups_train))
    r2_test_knn.append( r2_score(y_test, predicted_pickups))
    
    
print('R^2 Values for Train Using KNN Regression:', r2_train_knn)
print('\n')
print('R^2 Values for Test Using KNN Regression:', r2_test_knn)



R^2 Values for Train Using KNN Regression: [0.99999999826605368, 0.70705295578349214, 0.26384447277586165, 0.14062785261746957, 0.16815399746437676, 0.041790871252071704, 0.026784122297934254, 0.022334620712726183, 0.013199508562418361, 0.011615787116215581, 0.0092690893153853926, 0.0079270698554815278, 0.0074171109814979985, 0.0056618388771036976]


R^2 Values for Test Using KNN Regression: [-0.87195646189656983, -0.32627859453665375, -0.1542926908927027, -0.089021508792754611, -0.11809935938908334, -0.020776306046869086, -0.010578038605963069, 0.0031685923025387419, 0.0036867169843481928, 0.0026448150105176094, 0.0015801451321187931, 0.0015217888123526535, 0.00056359830487651141, -0.0006410793680169391]


In [25]:
# random forest regression
from sklearn.ensemble import RandomForestRegressor
r2_train_rf = []
r2_test_rf = []

# check multiple depths to see which depth is best
for i in range(1, 20):
    rf_reg = RandomForestRegressor(max_depth=i)
    rf_reg.fit(X_train, y_train)

    rf_yhat_train = rf_reg.predict(X_train)
    rf_yhat_test = rf_reg.predict(X_test)

    r2_train_rf.append( r2_score(y_train, rf_yhat_train))
    r2_test_rf.append( r2_score(y_test, rf_yhat_test))

print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf)

R^2 Values for Train Using Random Forest Regression: [0.49720296756434346, 0.61053350899380598, 0.67511950828551681, 0.67387294267734843, 0.73276489542719059, 0.76318669077294254, 0.82590310068220496, 0.89548927397264066, 0.82065384409508457, 0.89398625403987209, 0.82760577348070163, 0.79403836942059292, 0.8100563877259388, 0.82224424426050358, 0.85025771122325189, 0.85715368168184147, 0.9331919843632086, 0.80620122624416679, 0.94136130639871862]


R^2 Values for Test Using Random Forest Regression: [0.065688530132135381, 0.11955126119857296, 0.23689289946392322, 0.26268485949846965, 0.27210599525123269, 0.31376558972213309, 0.28322097770388877, 0.33781445372311392, 0.35981602438696281, 0.27767797065812116, 0.22661389303616364, 0.2564865245822131, 0.15968107555102107, 0.31209263225671557, 0.24284017786329626, 0.25838381619956918, 0.26517810613637338, 0.25505016476376241, 0.24039207539967111]


In [26]:
# get best depth
index, value = max(enumerate(r2_test_rf), key=operator.itemgetter(1))
best_depth2 = index+1
print('Best Depth for Random Forest Tree Depth:', best_depth2 )

Best Depth for Random Forest Tree Depth: 9


## Fine Tuning Phase

### Ridge and Lasso on Polynomial Term Regression

In [27]:
# Ridge Regression on Polynomial Term Regression
lambdas = [.001,.005,1,5,10,50,100,500,1000]    
ridge = RidgeCV(alphas=lambdas, fit_intercept=False, normalize=True, cv=10)
ridge.fit(X_poly_train_all, y_train)
print("Ridge train R2: ", ridge.score(X_poly_train_all, y_train))
print('Ridge test R2', ridge.score(X_poly_test_all, y_test))


Ridge train R2:  0.326729914555
Ridge test R2 0.257295436254


In [28]:
# Lasso Regression on Polynomial Term Regression
lasso = LassoCV(alphas=lambdas, fit_intercept=False, normalize=True, cv=10)
lasso.fit(X_poly_train_all, y_train)
print("Lasso train R2: ", lasso.score(X_poly_train_all, y_train))
print('Lasso test R2', lasso.score(X_poly_test_all, y_test))

C:\Users\IhnSik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso train R2:  0.330382769017
Lasso test R2 0.223520995991


C:\Users\IhnSik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [29]:
# Fine Tuning Random Forest: get R^2 values for optimal depth that we calculated in shotgun approach
r2_train_rf = []
r2_test_rf = []

rf_reg = RandomForestRegressor(max_depth=best_depth2)
rf_reg.fit(X_train, y_train)

rf_yhat_train = rf_reg.predict(X_train)
rf_yhat_test = rf_reg.predict(X_test)

r2_train_rf.append( r2_score(y_train, rf_yhat_train))
r2_test_rf.append(r2_score(y_test, rf_yhat_test))

print('R^2 Values for Train, Test Using Random Forest Regression:', r2_train_rf, r2_test_rf)

R^2 Values for Train, Test Using Random Forest Regression: [0.90189089522031818] [0.23157544697065435]


In [30]:
# step 1: fine tuning number of trees
r2_train_rf_trees = []
r2_test_rf_trees = []

# will try various number of trees
trees = [2**x for x in range(8)]  # 2, 4, 8, 16, 32, ... 

# try different trees with optimal depth
for n_trees in trees:
    rf = RandomForestRegressor(n_estimators=n_trees, max_depth=best_depth2, max_features='auto')
    rf.fit(X_train, y_train)

    rf_yhat_train = rf.predict(X_train)
    rf_yhat_test = rf.predict(X_test)

    r2_train_rf_trees.append(r2_score(y_train, rf_yhat_train))
    r2_test_rf_trees.append(r2_score(y_test, rf_yhat_test))
    
print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf_trees)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf_trees)

R^2 Values for Train Using Random Forest Regression: [0.58601731456312234, 0.80518079006581667, 0.74293370836669848, 0.73025122456148694, 0.90057120880801123, 0.84795860443696702, 0.88083329180459491, 0.85593486474751224]


R^2 Values for Test Using Random Forest Regression: [-0.013935959307974422, 0.07300285483685709, 0.14399150549967843, 0.31431646702194216, 0.24648467198446888, 0.30167909769024848, 0.26542634682886901, 0.26380393138249736]


In [31]:
# get best number of trees
index, value = max(enumerate(r2_test_rf_trees), key=operator.itemgetter(1))
best_trees2 = trees[index]
print('RandomForest Best Number of Trees to Use:',best_trees2)
print('R^2 Value:',value)

RandomForest Best Number of Trees to Use: 8
R^2 Value: 0.314316467022


In [32]:
# step 2: fine tuning number of predictors used
r2_train_rf_feat = []
r2_test_rf_feat = []
for i in range(1, 43):
    rf = RandomForestRegressor(n_estimators=best_trees2, max_depth=best_depth2, max_features=i)
    rf.fit(X_train, y_train)

    rf_yhat_train = rf.predict(X_train)
    rf_yhat_test = rf.predict(X_test)

    r2_train_rf_feat.append(r2_score(y_train, rf_yhat_train))
    r2_test_rf_feat.append(r2_score(y_test, rf_yhat_test))
    
print('R^2 Values for Train Using Random Forest Regression:', r2_train_rf_feat)
print('\n')
print('R^2 Values for Test Using Random Forest Regression:', r2_test_rf_feat)

R^2 Values for Train Using Random Forest Regression: [0.70232486693060547, 0.75232293394050664, 0.82584846666606015, 0.78871358857309493, 0.83202340852814349, 0.77036950971865881, 0.69750841053357981, 0.72221566935792425, 0.82605330010820466, 0.88908987028320463, 0.53399337078805409, 0.79383875094158807, 0.74279856771067865, 0.85108796833255596, 0.74823493502486782, 0.82653720416043419, 0.7730303767973743, 0.84385479353062809, 0.88503861073119205, 0.80246760369575221, 0.85823249778833055, 0.82757283843921481, 0.75650047731025472, 0.82466286040349956, 0.82286799297186164, 0.90786430348903646, 0.80296110444585667, 0.7732600660515343, 0.86276618975212127, 0.86030642846261174, 0.80603377507793905, 0.79419412234228093, 0.8796156487774599, 0.86094027884788926, 0.82723584845490716, 0.86540695672811618, 0.75646342389032162, 0.78299155506435558, 0.86664667606014179, 0.88392426278384006, 0.85124097497081319, 0.89521892610276799]


R^2 Values for Test Using Random Forest Regression: [0.0927173446

In [33]:
# get best number of predictors and best R^2 value
index, value = max(enumerate(r2_test_rf_feat), key=operator.itemgetter(1))
print('RandomForest Number of Predictors for Best value:',index+1)
print('Final Best R^2 Value:',value)

RandomForest Number of Predictors for Best value: 17
Final Best R^2 Value: 0.313345690369
